<a href="https://colab.research.google.com/github/prathyyyyy/nasdaq-stock-prediction/blob/main/1_nasdaq_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NASDAQ - National Association of Securities Dealers Automated Quotations**

## **Introduction:**

- **Nasdaq is one of the most prominent stock exchanges globally, recognized for its comprehensive electronic trading platform and a significant concentration of technology and biotech companies. Established on February 8, 1971, and headquartered in New York City, the Nasdaq operates entirely electronically, facilitating fast, efficient, and transparent trading. It boasts a substantial market capitalization, often rivaling that of the New York Stock Exchange (NYSE), and lists many of the world's leading tech giants, such as Apple, Microsoft, Amazon, and Google.**

- **The Nasdaq's trading hours extend beyond the regular session of 9:30 AM to 4:00 PM Eastern Time (ET) to include pre-market and after-hours trading, providing investors with extended opportunities to engage in the market. This flexibility, coupled with its electronic trading capabilities, makes Nasdaq a vital hub for innovation and a critical player in the global financial markets. The performance of its indices, particularly the Nasdaq-100, is widely regarded as a barometer of the health of the technology sector and overall investor sentiment towards growth stocks.**

# **NASDAQ - Stock Market Analysis Using Pyspark, Pytimetk, Plotly and grekite (Trends and Seasonality)**

1. **Pyspark for handling large csv file of NASDAQ data (Faster Processing of Dataset).**
2. **Plotly and PytimeTK for interactive charts to provide insights of NASDAQ dataset.**
3. **Linkedin Greykite an open source library to find Trends for Nasdaq Data.**

#### **[Link for NASDAQ,NYSE AND S&P500](https://www.kaggle.com/datasets/paultimothymooney/stock-market-data/)**

# **1. Import Module - Pyspark, Pandas, Pytimetk And Greykite**

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pytimetk as tk
import plotly.express as px
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
import plotly.figure_factory as ff
import plotly.graph_objects as go
from pyspark.sql.types import StructField, StructType, StringType, FloatType,IntegerType, DateType
from pyspark.sql.functions import (col, current_timestamp, to_timestamp, concat, lit,year, col,desc, sum as _sum, avg,
                                   input_file_name, regexp_extract, mean, stddev, min, max, count, date_format, to_date, last)
from plotly.subplots import make_subplots
from pytimetk import plot_timeseries

# **2. Start Spark Session**

In [2]:
spark = SparkSession.builder \
    .appName("NASDAQ-Time-Series-Analysis") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.debug.maxToStringFields", "200")\
    .getOrCreate()

# **3. Modify Schema and Import NASDAQ Data**

###  - **NASDAQ Has Approximately 9 Million Rows****

In [3]:
nasdaq_schema = StructType(
    fields=[
        StructField("Date", StringType(), True),
        StructField("Low",FloatType(),True),
        StructField("Open",FloatType(),True),
        StructField("Volume",IntegerType(),True),
        StructField("High",FloatType(),True),
        StructField("Close",FloatType(),True),
        StructField("Adjusted Close",FloatType(),True),
    ]
)

In [4]:
nasdaq_df = spark.read.csv("/content/drive/MyDrive/nasdaq/csv/",
                          header = True, schema = nasdaq_schema)

In [5]:
# Extract stock ticker from filename
nasdaq_df = nasdaq_df.withColumn("company_id", regexp_extract(input_file_name(), r"/([^/]+)\.csv", 1))\
                     .withColumnRenamed("Date","date")\
                     .withColumnRenamed("Low", "low")\
                     .withColumnRenamed("Volume", "volume")\
                     .withColumnRenamed("High", "high")\
                     .withColumnRenamed("Close", "close")\
                     .withColumnRenamed("Adjusted Close", "adjusted_close")

In [6]:
nasdaq_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- low: float (nullable = true)
 |-- Open: float (nullable = true)
 |-- volume: integer (nullable = true)
 |-- high: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adjusted_close: float (nullable = true)
 |-- company_id: string (nullable = false)



In [7]:
nasdaq_df.show()

+----------+--------+--------+------+--------+--------+--------------+----------+
|      date|     low|    Open|volume|    high|   close|adjusted_close|company_id|
+----------+--------+--------+------+--------+--------+--------------+----------+
|21-02-1973|0.395062|     0.0| 15188|0.395062|0.395062|      0.395062|      DIOD|
|22-02-1973| 0.37037|     0.0|  9113| 0.37037| 0.37037|       0.37037|      DIOD|
|23-02-1973|0.345679|     0.0|  3038|0.345679|0.345679|      0.345679|      DIOD|
|26-02-1973|0.345679|     0.0|  1519|0.345679|0.345679|      0.345679|      DIOD|
|27-02-1973|0.345679|     0.0| 29869|0.345679|0.345679|      0.345679|      DIOD|
|28-02-1973|0.345679|     0.0|  1519|0.345679|0.345679|      0.345679|      DIOD|
|01-03-1973|0.320988|     0.0| 18225|0.320988|0.320988|      0.320988|      DIOD|
|02-03-1973|0.320988|     0.0| 20250|0.320988|0.320988|      0.320988|      DIOD|
|05-03-1973|0.345679|     0.0|  6075|0.345679|0.345679|      0.345679|      DIOD|
|06-03-1973|0.32

# **4. statistical Analysis**

In [8]:
# Converting to Pyspark to pandas as it gives unclean view

summary_stats = nasdaq_df.describe().toPandas()
summary_stats

,summary,date,low,Open,volume,high,close,adjusted_close,company_id
0,count,8752326,8622050,8622048,7998947,8622048,8622049,8622049,8752326
1,mean,237000.0,4.215772190301433E9,4.324767882834021E9,1563230.5113445558,4.411166614741718E9,4.306316291726486E9,4.306316105220052E9,None
2,stddev,None,4.3471188152532007E11,4.4585405010880865E11,1.8027836197525736E7,4.5413892595984344E11,4.437969201593135E11,4.437969201611227E11,None
3,min,01-02-1971,0.0,0.0,0,0.0,0.0,-101.84761,AAL
4,max,31-12-2021,8.5239003E13,9.1249197E13,2127171200,9.1249197E13,8.6750999E13,8.6750999E13,ZUMZ


In [9]:
company_descriptive_stats = nasdaq_df.groupBy('company_id').agg(
    mean('close').alias('mean'),
    stddev('close').alias('stddev'),
    min('close').alias('min'),
    max('close').alias('max'),
    count('close').alias('count')).toPandas()

In [10]:
company_descriptive_stats

,company_id,mean,stddev,min,max,count
0,MAT,15.787576,11.190194,0.955733,47.820000,11724
1,AAPL,16.332145,34.928490,0.049107,182.009995,10590
2,OKE,19.257495,20.865788,1.067020,77.519997,10640
3,CINF,34.004954,29.580433,1.074084,141.250000,10778
4,KBAL,9.711870,4.283591,1.219750,21.889999,10778
...,...,...,...,...,...,...
1559,PERF,7.623600,1.332584,5.270000,10.990000,30
1560,LION,9.991843,0.089174,9.811000,10.170000,254
1561,ACHN,0.319588,0.111421,0.200000,0.510000,289
1562,LGCY,0.010725,0.002612,0.010000,0.020000,69


In [11]:
earliest_date = nasdaq_df.agg({"date": "min"}).collect()[0][0]
latest_date = nasdaq_df.agg({"date": "max"}).collect()[0][0]

print(f"Earliest Date Recorded: {earliest_date} and Last Date Recorded : {latest_date}")

Earliest Date Recorded: 01-02-1971 and Last Date Recorded : 31-12-2021


In [12]:
nasdaq_df = nasdaq_df.withColumn("date", to_date(col("Date"), "dd-MM-yyyy"))

# **5. Company Stock Volume And Adjusted Closing Analysis**

In [13]:
def stock_volume(company_id,company_name, year_val):

    filtered_df = nasdaq_df.filter((nasdaq_df['company_id'] == company_id) & (year(nasdaq_df['date']) == year_val))

    filtered_pd_df = filtered_df.toPandas()

    filtered_pd_df['date'] = pd.to_datetime(filtered_pd_df['date'])

    fig = tk.plot_timeseries(
        data=filtered_pd_df,
        date_column='date',
        value_column='volume',
        title=f'{company_name} Stock Volume in {year_val} - NASDAQ')

    fig.show()

In [14]:
stock_volume("NVDA", "NVIDIA", 2021)

In [15]:
stock_volume("MSFT", "Microsoft", 2010)

In [16]:
def closing_price(company_id,company_name, year_val):

    filtered_df = nasdaq_df.filter((nasdaq_df['company_id'] == company_id) & (year(nasdaq_df['date']) == year_val))

    filtered_pd_df = filtered_df.toPandas()

    filtered_pd_df['date'] = pd.to_datetime(filtered_pd_df['date'])

    fig = tk.plot_timeseries(
        data=filtered_pd_df,
        date_column='date',
        value_column='adjusted_close',
        title=f'{company_name} Closing Price in {year_val} - NASDAQ')

    fig.show()

In [17]:
closing_price("AMD", "Advanced Micro Devices - AMD", 2020)

# **6. Stock Trend Detection Using Greykite**

In [18]:
def helper_func(company_id, company_name, year_val):

    filtered_df = nasdaq_df.filter(
        (nasdaq_df['company_id'] == company_id) & (year(nasdaq_df['date']) == year_val))

    filtered_pd_df = filtered_df.toPandas()

    filtered_pd_df['date'] = pd.to_datetime(filtered_pd_df['date'])

    filtered_pd_df.rename(columns={'date': 'ds', 'volume': 'y'}, inplace=True)

    return filtered_pd_df

In [19]:
def stock_vol_trend(df):
    if df is None or df.empty:
        raise ValueError("The input DataFrame is empty or None.")

    model = ChangepointDetector()
    res = model.find_trend_changepoints(
        df=df,
        time_col="ds",
        value_col="y",
        yearly_seasonality_order=3,
        regularization_strength=0.5,
        resample_freq="1M",
        potential_changepoint_n=10,
        yearly_seasonality_change_freq="365D",
        no_changepoint_distance_from_end="30D"  # Slightly less to allow change near the end
    )

    fig = model.plot(
        observation=True,
        trend_estimate=False,
        trend_change=True,
        yearly_seasonality_estimate=False,
        adaptive_lasso_estimate=True,
        plot=False
    )

    fig.update_layout(height=800, width=1400, title="Stock Volume with Detected Trend Change Points")
    fig.show()

In [20]:
filtered_data = helper_func("AMZN", "Amazon", 2015)
stock_vol_trend(filtered_data)

In [21]:
def helper_func2(company_id, company_name, year_val):

    filtered_df = nasdaq_df.filter((nasdaq_df['company_id'] == company_id) & (year(nasdaq_df['date']) == year_val))

    filtered_pd_df = filtered_df.toPandas()

    filtered_pd_df['date'] = pd.to_datetime(filtered_pd_df['date'])

    filtered_pd_df.rename(columns={'date': 'ds', 'adjusted_close': 'y'}, inplace=True)

    return filtered_pd_df

# **7. Analysis of Top 10 Performers provided by NASDAQ**

## - **NVIDIA, AMD and Apple has been the top performer of year 2021 due to crypto mining, boom in AI and Stock of apple has went high due to their sales.**

In [22]:
def adj_close_trend(df):
    if df is None or df.empty:
        raise ValueError("The input DataFrame is empty or None.")

    df['ds'] = pd.to_datetime(df['ds'])
    df = df.reset_index(drop=True)

    model = ChangepointDetector()
    res = model.find_trend_changepoints(
        df=df,
        time_col="ds",
        value_col="y",
        yearly_seasonality_order=3,
        regularization_strength=0.5,
        resample_freq="1M",
        potential_changepoint_n=10,
        yearly_seasonality_change_freq="365D",
        no_changepoint_distance_from_end="30D"
    )

    fig = model.plot(
        observation=True,
        trend_estimate=False,
        trend_change=True,
        yearly_seasonality_estimate=False,
        adaptive_lasso_estimate=True,
        plot=False
    )

    fig.update_layout(
        height=800,
        width=1400,
        title="Adjusted Close Price with Detected Trend Change Points",
        xaxis_title="Date",
        yaxis_title="Adjusted Close Price"
    )
    fig.show()

In [23]:
filtered_data = helper_func2("AMZN", "Amazon", 2015)
adj_close_trend(filtered_data)

In [24]:
year_df = nasdaq_df.filter(year(col("date")) == 2021).toPandas()

year_df['date'] = pd.to_datetime(year_df['date'])

year_df = year_df[year_df['date'].dt.year == 2021]

volume_by_id = year_df.groupby('company_id')['volume'].sum()
top_10_ids = volume_by_id.nlargest(10).index
filter_df = year_df[year_df['company_id'].isin(top_10_ids)]
pivot_df = filter_df.pivot(index='date', columns='company_id', values='close')

fig = make_subplots(rows=1, cols=1)


for column in pivot_df.columns:
    fig.add_trace(
        go.Scatter(x=pivot_df.index, y=pivot_df[column], name=column),
        row=1, col=1
    )


fig.update_layout(
    title_text='Time Series of Closing Prices for Top 10 Companies by Volume Traded in 2021',
    xaxis_title='Date',
    yaxis_title='Closing Price',
    legend_title='Company ID',
    showlegend=True,
    width = 1300,
    height = 1000
)

fig.show()

In [25]:
filtered_df = year_df[year_df['company_id'].isin(top_10_ids)]

fig = make_subplots()

for company_id in filtered_df['company_id'].unique():
    subset_df = filtered_df[filtered_df['company_id'] == company_id]
    fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['adjusted_close'], mode='lines', name=company_id))

fig.update_layout(
    title='Adjusted Closing Prices for Top 10 Companies',
    xaxis_title='Date',
    yaxis_title='Adjusted Closing Price',
    xaxis=dict(tickformat='%Y-%m-%d', tickangle=45),
    legend_title_text='Company ID',
    hovermode='x unified',
    template='plotly_white',
)

fig.show()

In [26]:
volatility = pivot_df.std().sort_values(ascending=False)

fig = px.bar(volatility,
             x=volatility.index,
             y=volatility.values,
             labels={'y': 'Standard Deviation', 'x': 'Ticker'},
             title='Volatility of Closing Prices (Standard Deviation)')

fig.show()

In [27]:
correlation_matrix = pivot_df.corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='tealgrn',
    colorbar=dict(title='Correlation')
))

fig.update_layout(
    title='Correlation Matrix of Closing Prices',
    xaxis_title='Ticker',
    yaxis_title='Ticker',
    width = 1400,
    height = 1000
)
fig.show()

In [28]:
perc_change = ((pivot_df.iloc[-1] - pivot_df.iloc[0]) / pivot_df.iloc[0]) * 100

fig = px.bar(perc_change,
             x=perc_change.index,
             y=perc_change.values,
             labels={'y': 'Percentage Change (%)', 'x': 'Ticker'},
             title='Percentage Change in Closing Prices')
fig.show()

In [29]:
daily_return = pivot_df.pct_change().dropna()

avg_daily_return = daily_return.mean()
risk = daily_return.std()

risk_return_df = pd.DataFrame({'Risk': risk, 'Average Daily Return': avg_daily_return})

fig = px.scatter(risk_return_df, x="Risk", y="Average Daily Return", text=risk_return_df.index)

fig.update_traces(
    marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),
    textposition="top center"
)

fig.update_layout(
    title="Risk vs. Return Analysis",
    xaxis_title="Risk (Standard Deviation)",
    yaxis_title="Average Daily Return",
    showlegend=False
)

# Display the plot
fig.show()